In [9]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Looking in indexes: https://download.pytorch.org/whl/cu118


In [1]:
# !pip install transformers
!pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.4/409.4 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalling requests-toolbelt-0.10.1:
      Successfully uninstalled requests-toolbelt-0.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.3 requires cubinlinker, which is not installed.
cudf 24

In [10]:
# Import necessary modules
from transformers import pipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [16]:
# Load the summarization pipeline
def load_summarizer(model_name="facebook/bart-large-cnn"):
    """
    Load the summarization model pipeline.
    Args:
        model_name (str): The name of the Hugging Face model.
    Returns:
        summarizer pipeline object
    """
    
    summarizer = pipeline("summarization", model=model_name, device=0)
    return summarizer


In [17]:
def split_text_with_langchain(text, chunk_size=4096, chunk_overlap=200):
    """
    Splits the text into manageable chunks using LangChain's RecursiveCharacterTextSplitter.
    Args:
        text (str): The text to split.
        chunk_size (int): Maximum size of each chunk in tokens.
        chunk_overlap (int): Number of overlapping characters between chunks.
    Returns:
        List of text chunks.
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""]
    )
    chunks = text_splitter.split_text(text)
    return chunks

In [18]:
# Function to split long text into manageable chunks
def split_text(text, max_chunk_size=1000):
    """
    Splits the text into smaller chunks to handle models with token limits.
    Args:
        text (str): Input text to summarize.
        max_chunk_size (int): Maximum chunk size (tokens).
    Returns:
        List of text chunks.
    """
    sentences = text.split('. ')
    chunks, current_chunk = [], ""
    for sentence in sentences:
        if len(current_chunk) + len(sentence) + 1 <= max_chunk_size:
            current_chunk += sentence + ". "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + ". "
    chunks.append(current_chunk.strip())
    return chunks


In [21]:

# Function to summarize text
def summarize_text(file_path, model_name="facebook/bart-large-cnn"):
    """
    Summarizes the content of a given .txt file.
    Args:
        file_path (str): Path to the .txt file.
        model_name (str): Hugging Face model for summarization.
    Returns:
        Summary as a string.
    """
    # Load summarization model
    summarizer = load_summarizer(model_name)

    # Read text from the file
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()

    # Split text if it's too long
    max_chunk_size = 4096  # Model token limit
    chunks = split_text_with_langchain(text, chunk_size=4096, chunk_overlap=200)

    # Summarize each chunk
    summaries = []
    for chunk in chunks:
        summary = summarizer(chunk, max_length=130, min_length=30, do_sample=False)[0]["summary_text"]
        print("Chunk: ",chunk," - - - - ","Summary: ", summary, "\n\n")
        summaries.append(summary)
    
    # Combine all summaries
    final_summary = " ".join(summaries)
    return final_summary


In [22]:
# Path to the .txt file in your Kaggle environment
input_file = "/kaggle/input/map-reduce/mapreduce_osdi04.txt" 

# Choose the model (BART is used here, you can try T5 or PEGASUS as well)
model_name = "facebook/bart-large-cnn"

# Summarize the text
try:
    summary = summarize_text(input_file, model_name)
    print("Summary of the Document:")
    print(summary)
except Exception as e:
    print(f"An error occurred: {e}")


Chunk:  MapReduce: Simplied Data Processing on Large Clusters
Jeffrey Dean and Sanjay Ghemawat
jeff@google.com, sanjay@google.com
Google, Inc.
Abstract
MapReduce is a programming model and an associ-
ated implementation for processing and generating large
data sets. Users specify a map function that processes a
key/value pair to generate a set of intermediate key/value
pairs, and a reduce function that merges all intermediate
values associated with the same intermediate key. Many
real world tasks are expressible in this model, as shown
in the paper.
Programs written in this functional style are automati-
cally parallelized and executed on a large cluster of com-
modity machines. The run-time system takes care of the
details of partitioning the input data, scheduling the pro-
gram's execution across a set of machines, handling ma-
chine failures, and managing the required inter-machine
communication. This allows programmers without any
experience with parallel and distributed systems t

/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1284: indexSelectLargeIndex: block: [212,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1284: indexSelectLargeIndex: block: [212,0,0], thread: [1,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1284: indexSelectLargeIndex: block: [212,0,0], thread: [2,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1284: indexSelectLargeIndex: block: [212,0,0], thread: [3,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1284: indexSelectLargeIndex: block: [212,0,0], thread: [4,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1284: indexSelectLargeIndex: block: [212,0,0], thread: [5,0,0] Assertion `srcIn